In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/KimDongyoung/Desktop/my_git/mygit/DAT/24-2/프로젝트/data/recipe.csv', encoding='cp949', encoding_errors='ignore')
df.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,10072,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,skfo0701,꽃날,5817,3,27,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,20070402205937
2,128932,입안에서 톡톡톡,알밥,skfo0701,꽃날,6975,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,20070402224355
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,3339,0,11,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,20070410142301
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,7173,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,20070501000844


In [3]:
len(df['CKG_NM'].unique())

49680

In [4]:
df = df.drop_duplicates(subset='CKG_NM', keep='first')
len(df)

49680

In [5]:
import requests, json
from bs4 import BeautifulSoup

def food_info(name):
    '''
    This function gives you food information for the given input.

    PARAMETERS
        - name(str): name of Korean food in Korean ex) food_info("김치찌개")
    RETURN
        - res(dict): dict containing info for some Korean food related to 'name'
            - res['name'](str): name of food
            - res['ingredients'](str): ingredients to make the food
            - res['recipe'](list[str]): contain recipe in order
    '''
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else: 
        print("HTTP response error :", response.status_code)
        return
    
    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    if not food_list:
        print(f"No results found for {name}")
        return
    
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else: 
        print("HTTP response error :", new_response.status_code)
        return
    
    food_info = soup.find(attrs={'type':'application/ld+json'})
    
    if food_info is None:
        print(f"No structured data found for {name}")
        return
    
    try:
        result = json.loads(food_info.text)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return

    # 'recipeIngredient'와 'recipeInstructions'가 있는지 확인
    ingredient_list = result.get('recipeIngredient', None)
    if ingredient_list:
        ingredients = ', '.join(ingredient_list)
    else:
        ingredients = 'No ingredients available'
    
    recipe_list = result.get('recipeInstructions', None)
    if recipe_list:
        recipe = [step.get('text', 'No instructions available') for step in recipe_list]
        recipe = [f'{i+1}. {step}' for i, step in enumerate(recipe)]
    else:
        recipe = ['No recipe available']
    
    res = {
        'name': name,
        'ingredients': ingredients,
        'recipe': recipe
    }
    
    return res

In [6]:
df['CKG_STA_ACTO_NM'].value_counts()

CKG_STA_ACTO_NM
일상      24838
간식       9018
초스피드     3549
손님접대     2759
영양식      2255
술안주      1896
다이어트     1555
도시락      1135
기타        914
명절        498
야식        346
이유식       310
해장        227
Name: count, dtype: int64

In [8]:
filtered_df = df[['CKG_STA_ACTO_NM', 'CKG_NM']]
filtered_df.head()

,CKG_STA_ACTO_NM,CKG_NM
0,간식,어묵김말이
1,일상,두부새우전
2,일상,알밥
3,일상,현미호두죽
4,술안주,북어갈비


In [13]:
results = []


for food_name in filtered_df['CKG_NM'].iloc[0:200]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df = pd.DataFrame(results)

In [14]:
result_df

,name,ingredients,recipe
0,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...","[1. 파프리카와 양파는 채썰어 살짝 볶아둔다., 2. 당면은 3분정도 삶아 물기를..."
1,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",[1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간...
2,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","[1. '날치알' 하나면 영양&비주얼 동시 캐치!, 2. 단무지, 맛살을 잘게 다져..."
3,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간","[1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다., 2. 호두는 끓는 물..."
4,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",[1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고...
...,...,...,...
195,충무김밥,"오징어 몸통 1마리, 오징어 다리 2마리, 오뎅 4장, 단무지 1/2개 (중간크기)...","[1. 오뎅을 큼직하게 6등분해 주세요., 2. 팬에 앞뒤로 노릇노릇 한번 볶아 주..."
196,단호박찜케익,"박력쌀가루 140g (또는 박력분), 호박가루 7g, 설탕 30g, 계란 1개, 우...",[1. 먼저 계란을 풀고 체친 박력쌀가루와 모든재료를 다 넣고 잘 섞어줍니다. (너...
197,연어간장조림,"연어 200g, 정종, 소금, 후추, 간장 1.5T, 올리고당 1T, 물 100ml...","[1. 연어를 먹기좋은 사이즈로 등분합니다., 2. 연어에 소금, 후추, 정종 1/..."
198,햄크림파스타,"스파게티 면 적당량, 햄 180g, 마늘 5개, 양파 1/2개, 생크림 200ml ...","[1. 먼저 웍에 물을 적당히 채운뒤 굵은 소금을 넣고 끓여 준다., 2. 끓기 시..."


In [16]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[201:400]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df2 = pd.DataFrame(results)

No structured data found for 식빵
JSON parsing error: Invalid control character at: line 5 column 33 (char 96)
JSON parsing error: Invalid control character at: line 5 column 26 (char 89)
No structured data found for 노오븐마늘빵


In [17]:
result_df2

,name,ingredients,recipe
0,느타리버섯구이,"느타리버섯 1그릇, 고추장 1큰술, 고춧가루 1큰술, 다진 마늘 1작은술, 다진 파...",[1. 느타리 버섯은 작은것은 그대로 쓰시고 큰것은 작은것 크기로 찢어 준비 한 후...
1,딸기가나슈케이크,"생크림 200g, 딸기, 장식용 초코렛 조금, 박력분 75g, 버터 20g, 설탕 ...","[1. 계란 전란을 볼에 넣고 휘핑기로 휘핑하다 설탕을 넣고 휘핑해 줍니다., 2...."
2,베이컨,"베이컨 적당량, 물 적당량, 키친타올 적당량","[1. 달궈진 팬에 베이컨을 올린다., 2. 베이컨이 잠길 정도로 물을 붓는다., ..."
3,떡볶이,"떡볶이떡 2컵, 물 2컵, 대파 1/2대, 통깨 약간, 고추장 2T, 고추가루 1....","[1. 먼저 종이컵 기준 물 2컵에 떡볶이떡을 넣고 센불에서 팔팔 끓여 줍니다., ..."
4,쌈밥,"밥 1/2공기, 리챔 1개, 신김치잎부분 1쪽, 부추 5개",[1. 신김치는 잎부분만 준비하고 양념을 씻은 다음 물기를 꼭 짜서 준비하세요. 부...
...,...,...,...
190,크림치즈머핀,"버터 100g, 크림치즈 190g, 계란 3개, 박력분 200g, 베이킹 파우더 3...","[1. 실온의 말랑 버터와 크림치즈를 한 볼에 넣고 거품기로 풀어준 다음 설탕, 소..."
191,마늘쫑볶음,"마늘쫑 1묶음, 볶은깨 톡톡, 식용유 2T, 간장 3T, 올리고당 1.5T, 맛술 ...","[1. 먹기좋게 자른 마늘종을 깨끗이 씻어 물기를 빼주세요., 2. 볶음팬에 식용유..."
192,굴생채,"굴 2봉지 (싱싱한), 배 1개, 고춧가루 2큰술, 간장 2큰술, 황설탕 1작은술,...","[1. 굴은 옅은 소금물에 살살 씻어 체에 밭쳐 물기를 제거해주세요., 2. 배는 ..."
193,국밥,"시판 순대 4컵, 대파 1대, 청양고추 2개, 시판 곰탕 1봉지 (600g), 물 ...","[1. 그릇에 양념장을 만들어주세요. 다진 마늘 1.5, 들깨가루 2.5, 고춧가루..."


In [19]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[401:600]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df3 = pd.DataFrame(results)

In [20]:
result_df3

,name,ingredients,recipe
0,쥐포볶음,"쥐포 10장, 식용류 1T, 고춧가루 1T, 고추장 1T, 설탕 1T, 간장 1T,...",[1. 쥐포는 마른팬에 앞뒤로 살짝 구워주세요(제 쥐포는 일반쥐포보다 큰 사이즈랍니...
1,오삼불고기,"오징어 1마리, 삼겹살 250g, 양파 1/2개, 깻잎 5장, 대파 약간, 설탕 2...","[1. 삽겹살,오징어를 적당한 크기로 잘라 준비해 줍니다., 2. 양념장을 분량대로..."
2,피망잡채,No ingredients available,[No recipe available]
3,연포탕,"낙지 4마리, 무 1/5개 (100g), 미나리 1줌 (50g), 표고버섯 3개, ...",[1. 무는 나박 썰고 표고버섯은 채 썰고 청양 고추는 송송 썰고 대파는 어슷 썰고...
4,소고기콩나물두부국,"소고기 100g, 콩나물 200g, 두부 1/2모, 청양고추 2개, 다진마늘 1큰술...","[1. 콩나물을 씻은뒤 물을 넣어 삶아줍니다., 2. 삶을때 다진마늘 1큰술도 같이..."
...,...,...,...
194,커피빙수,"얼음, 아이스크림 약간, 에스프레소 1잔, 각설탕 1개, 빙수팥 약간, 연유",[1. 일반 빙수를 먹을땐 우유+물을 1:1.3 비율로 얼린 우유를 사용하지만 커피...
195,두부감자채탕수,"두부 2모, 감자 1개, 당근 1/3개, 빵가루 2컵, 오이1/2개, 계란 1개, ...",[1. 소스를 먼저 준비할께용. 튀김요리는 바로바로 먹어야 맛이 있으니 튀겨내자 마...
196,감자주먹밥구이,"밥 1공기, 감자 1개, 김치 1줌, 설탕 1/2큰술, 들기름 1/2큰술, 참기름 ...","[1. 쌀을 씻은 후 감자를 넣고 밥을 짓는다., 2. 김치를 잘게 썰어서 들기름과..."
197,옥수수맛탕,"옥수수, 밀가루, 버터, 식용유, 꿀","[1. 옥수수를 한알한알 따서 준비해 주세요. (옥수수캔 사용가능), 2. 옥수수 ..."


In [21]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[601:900]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df4 = pd.DataFrame(results)

JSON parsing error: Invalid control character at: line 5 column 27 (char 90)


In [22]:
result_df4

,name,ingredients,recipe
0,새우구이,"자숙새우 2줌, 식용유 약간, 다진마늘 1큰술, 버터 1/2큰술, 소금 1꼬집, 파...","[1. 칵테일 새우는 물에 한번 씻어서 물기를 빼주었구요, 2. 식용유를 살짝 두른..."
1,매쉬드포테이토,"감자 2개, 버터 70g, 우유 30g, 소금, 후추, 파슬리","[1. 재료를 준비 해주세요., 2. 껍질을 벗긴 감자는 익히기 쉽도록 작게 썰어주..."
2,연어구이,"연어 1/2마리 (500g), 올리브유 2큰술, 허브솔트 1작은술, 마요네즈 3큰술...",[1. 깨끗하게 손질되어 냉동한 연어 1/2마리...500g을 냉장실에서 해동을 해...
3,크림참치베이글샌드위치,"베이글 1개, 크림치즈 40g, 작은 참치캔 1/2개, 상추등야채, 피클","[1. 기름기를 뺀 참치에 말랑한 크림치즈를 넣어 잘 믹스해주세요, 2. 베이글을 ..."
4,닭가슴살겨자냉채,"양파 1/4개, 오이 1/3개, 파프리카 1/2개, 당근 1/2개, 깻잎 10장, ...",[1. 냉동닭가슴살을 이용했으며 한덩어리만 사용했어요 칼집을 내주고 삶으면 시간단...
...,...,...,...
293,두부탕,"순두부, 대파 1/4개, 양파 1/4개, 애호박 1/4개, 청양고추 1개, 달걀 1...","[1. 제일 먼저 멸치 육수를 끓여주세요. 멸치외에 있으면 표고버섯 약간, 다시마 ..."
294,커스터드크림,"우유 200g, 밀가루 20g, 달걀노른자 2개, 설탕 50g, 버터 20g, 바닐...","[1. 냄비에 모든 재료를 넣는다., 2. 거품기로 재료를 잘 섞는다., 3. 약불..."
295,요구르트스콘,"밀가루박력분 150g, 설탕 20g, 소금 1/2ts, 베이킹파우더 3g, 베이킹소...","[1. 먼저 박력분과 설탕,소금,b.p(베이킹파우더),b.s(베이킹소다)는 체쳐 준..."
296,청양고추볶음밥,"밥 2공기, 닭가슴살 1덩이, 칵테일새우 7마리, 청양고추 2개, 홍고추 1개, 베...",[1. 볶음밥에 들어갈 재료들을 잘게 잘라 주세요. 너무 다지지는 마시고 적당히 씹...


In [24]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[901:1200]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df5 = pd.DataFrame(results)

In [25]:
result_df5

,name,ingredients,recipe
0,치즈떡볶이,"떡볶이 떡 3컵, 어묵 2장, 비엔나소세지 5개, 양파 1/3개, 모차렐라치즈 1컵...","[1. 떡볶이에 들어갈 재료인 어묵, 양파, 소시지, 파를 먹기 좋은 크기로 썰어주..."
1,모카초코칩쿠키,"실온의녹인무염버터 60g, 황설탕 70g, 백설탕 15g, 달걀 25g, 꿀 5g,...","[1. 실온의 녹인 버터를 푼 후 황설탕과 백설탕을 넣고 섞는다., 2. 달걀을 2..."
2,고구마몽블랑케이크,"고구마 큰거 3개 (약800g), 계란 1개, 생크림 1/2컵, 설탕 2큰술, 버터...",[1. 고구마는 잘 씻어서 물에 푸욱 삶거나 쪄서 뜨거울때 껍질을 제거 하고 장식으...
3,단호박 밤머핀,"박력쌀가루 300g, 버터 150g, 설탕 220g, 계란 4개, 생크림 100cc...","[1. 먼저 계란을 잘 풀어놓고.., 2. 설탕과 버터를 믹싱하면서 계란물을 조금씩..."
4,배샤베트,No ingredients available,"[1. 배는 껍질과 속을 깎아서 준비하고요.. 계피가루와 꿀, 레몬즙을 준비했어요...."
...,...,...,...
294,쇠고기두부덮밥,"쇠고기(한우)불고기용 100g, 두부 1/4모, 새송이버섯 1개, 당근 1/4개, ...","[1. 쇠고기는 미리 분량의 재료대로 밑간을 해서 재워두세요., 2. 양파+새송이버..."
295,쇠고기굴소스볶음,"쇠고기 300g (한우), 쥬키니호박 1/3개, 홍파프리카 1/2개, 마늘 4알, ...",[1. 쇠고기는 핏물을 제거한뒤 먹기좋게 썰어 청주1큰술과 허브솔트+후춧가루로 미리...
296,불고기버거,"햄버거용빵 4개, 시판용 떡갈비 4장, 양상추 4장, 마요네즈 취향껏, 간장 10c...","[1. 양상추는 한장한장 뜯어서 물에 씻은다음 물기를 빼주세요,, 2. 양념레시피를..."
297,블루베리팬케이크,"핫케이크가루 250g, 우유 170ml, 블루베리 적당량, 계란 1개","[1. 블루베리를 식초 탄 물에 잘 씻어 두세요, 2. 계란1개를 풀어주세요, 3...."


In [26]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[1201:1500]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df6 = pd.DataFrame(results)

JSON parsing error: Invalid control character at: line 5 column 28 (char 91)


In [27]:
result_df6 

,name,ingredients,recipe
0,딸기잼쿠키,"무염버터 100g, 설탕 60g, 노른자 1개, 박력분 145g, 옥수수전분 5g,...",[1. 볼에 말랑한 버터를 넣고 부드럽게 풀어준 다음 설탕을 넣고 충분히 섞어주세요...
1,동부묵,"동부가루 30g, 물 400cc, 호박가루, 녹차가루, 파프리카가루, 진간장, 다진...","[1. 동부가루를 30~31g 정도 전자저울에서 재었어요., 2. 물 400cc(종..."
2,부추고기만두,"돼지고기 1kg, 부추 1단, 양파 1-1/2개, 대파 4-5개, 다진마늘 2스푼,...","[1. 부추,대파,양파를 잘게 다집니다., 2. 돼지고기에 소금,후추,생강술로 밑간..."
3,단호박푸딩,"우유 300ml, 생크림 100ml, 단호박 으깬것 350g , 설탕 70g , 바...",[1. 먼저 카라멜을 만들어 주세요. 냄비에 설탕을 넣고 약한불로 끓여 가장자리부터...
4,소시지달걀말이,"소시지 바 2개, 달걀 5개, 슬라이스치즈 2장, 다진당근 1숟가락, 다진대파 1숟...","[1. 소시지바는 꼬치를 빼 준비한다., 2. 볼에 달걀을 풀고 다진당근, 다진대파..."
...,...,...,...
293,야채케이크,"쌀가루박력분 160g, 양파 1/2개, 콘옥수수 4큰술, 슬라이스햄 4장, 청양고추...",[1. 양파를 썰어주실 때 양파의 씹는맛을 더 느끼고 싶으시면 적당하게 썰어주셔도 ...
294,삼겹살김치볶음밥,"삼겹살 2줄, 김치 2/3밥공기, 대파 10cm , 마늘 4쪽, 밥 1.5공기 , ...",[1. (2인분 기준 입니다^~^) 대파10센티는 송송~ 썰어 주시구요~ 마늘 4쪽...
295,모카비스킷,"박력분 200g, 무염버터 40g, 설탕 30g, 소금 2g, 달걀 1개 50g, ...",[1. 먼저 볼에 초콜릿을 넣고 뜨거운물에 중탕하여 녹인 후 따뜻하게 데운 우유와 ...
296,고구마케익,"고구마 4개 (150g), 달걀흰자 3개, 우유 2스푼","[1. 찐고구마,군고구마 등을 으깨어주세요, 2. 달걀 흰자를 이용해 머랭을 만들어..."


In [29]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[1501:2000]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df7 = pd.DataFrame(results)

No structured data found for 계란야채부침
JSON parsing error: Invalid control character at: line 5 column 21 (char 84)


In [30]:
result_df7

,name,ingredients,recipe
0,야채계란빵,"핫케이크가루 1컵, 우유 1/2컵, 달걀 1개, 달걀 8개, 고구마 1/2개 (70...",[1. 먼저 고구마(1/2개=70g)는 깨끗이 씻어 껍질째 아주 잘게 썰어 물에 5...
1,초코칩크림치즈컵케이크,"버터 80g, 황설탕 70g, 계란 1개, 박력쌀가루 120g, 코코아파우더 12g...",[1. 실온상태에 보관해둔 말랑말랑한 버터에 설탕을 조금씩 넣어가며 고속 휘핑해서 ...
2,미니불고기햄버거,"쇠고기불고기감 200g, 양파 1/4개, 슬라이스치즈 2장, 모닝빵 7개, 양상추 ...","[1. 쇠고기는 우리한우로 준비해서 핏물을 제거해주세요., 2. 핏물을 제거한 쇠고..."
3,견과류고구마쿠키,"무염버터 100g, 황설탕 50g, 꿀10g, 소금 1g, 달걀 1개, 박력분 13...",[1. 먼저 고구마(200g)는 깨끗이 씻어 껍질째 손톱크키만큼 네모나게 썰어 10...
4,까르보나라떡볶이,"떡볶이떡, 양파 1/2개, 베이컨 3줄, 브로컬리 조금, 계란 노른자 1개, 다진마...","[1. 떡을 제외한 속재료는 베이컨, 양파, 브로컬리에요. 새우나, 해물종류를 넣어..."
...,...,...,...
492,닭모래집튀김,"닭똥집 300g, 통마늘 1종이컵, 청량고추 5개, 우유 2종이컵, 굵은소금 1종이...","[1. 닭똥집 300g을 준비합니다., 2. 흐르는 물에 한번 깨끗이 씻어줍니다.,..."
493,가라아게,"닭다리살 200g, 녹말가루 3큰술, 식용유 넉넉하게, 달걀 푼것 1/2개, 간장 ...","[1. 닭다리살 200g은 껍질을 바닥으로 두고 한입크기로 썰어주세요., 2. 청주..."
494,불고기라이스크로켓,"다진 쇠고기 150g, 양파 1/2개, 당근 1/5개, 대파 1/2뿌리, 밥 2공기...","[1. 먼저 다진 쇠고기에 양념장 재료를 섞어 재워두고 양파, 당근, 대파는 잘게 ..."
495,초코퍼지컵케이크,"버터 80g, 다크 초콜릿 200g, 설탕 100g, 소금 2g, 달걀 2개, 박력...","[1. 먼저 박력분(75g), b.p(1/2t), 코코아 가루(15g)는 2번정도 ..."


In [31]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[2001:2500]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df8 = pd.DataFrame(results)

JSON parsing error: Invalid control character at: line 5 column 31 (char 94)
JSON parsing error: Expecting ',' delimiter: line 23 column 3053 (char 3859)


In [32]:
result_df8

,name,ingredients,recipe
0,초계탕,"닭 1마리 (9호), 통마늘 5개, 양파 작은 거 1개, 대파 2대, 물 17컵, ...",[1. 닭은 흐르는 물에 깨끗이 씻어 준비하고 냄비에 물 17컵을 넣고 (생수 3...
1,코코넛비스킷,"박력분 300g (or 중력분), 베이킹파우더 4g, 소금 4g, 설탕 30g, 버...","[1. 박력분 or 중력분, 베이킹파우더를 체치고 소금, 설탕과 함께 볼에 넣어주세..."
2,버섯양파샌드위치,"식빵 4장, 애느타리버섯 1팩, 양파 중 1개, 체다슬라이스 치즈 2장, 마요네즈 ...",[1. 먼저 양파 중 1개는 굵직하게 채 썰어 올리브오일 1을 두르고 달달 볶아줍니...
3,버섯샐러드,"버섯 3-4컵 (한입 크기로 썰어서), 마늘 1개 (다진 것), 래디쉬 2-3알 (...",[1. 소스의 재료를 섞어 데리야끼 소스를 만들어 둔다. 생강가루 대신 생강을 쓸 ...
4,계란쿠키,"버터 70g, 설탕 65g, 달걀 1개, 달걀 노른자 1개, 바닐라 오일 3~4방울...",[1. 버터를 깍둑썰기해서 볼에 담아 실온에 뒀다가 크림화한 뒤에 설탕을 2번 정도...
...,...,...,...
492,생선살튀김볼,"흰살생선 1쪽, 칵테일새우 1컵, 모듬해물 1/2컵, 모듬야채 1/2컵, 양파 1/...","[1. 냉동시켜논 생선살은 반쯤 녹았을때 작게 잘라서 분쇄기에 갈아줬습니다., 2...."
493,낙지스파게티,"바릴라스파게티 300g, 낙지 2마리, 토마토 1개, 마른 고추 1개, 마늘 6쪽,...","[1. 낙지는 깨끗이 씻어 먹기 좋은 크기로 썬다., 2. 토마토는 + 칼집을 내어..."
494,돼지고기감자조림,"돼지고기안심 300g, 감자 3개 (중간크기), 당근 4-5cm, 양파 1/2개, ...","[1. 돼지고기 안심은 핏물을 닦아내고, 감자와 비슷한 크기로 네모모양으로 잘라주세..."
495,가지조림,"가지 400g, 느타리버섯 120g, 마늘 20개, 홍고추 1개, 진간장 2큰술, ...",[1. 가지 2개 준비했어요. 깨끗이 씻은 가지는 2센치 두께로 썰어주세요. 제가 ...


In [34]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[2501:3000]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df9 = pd.DataFrame(results)

JSON parsing error: Expecting ',' delimiter: line 23 column 1175 (char 1843)
JSON parsing error: Invalid control character at: line 5 column 33 (char 96)


In [35]:
result_df9

,name,ingredients,recipe
0,뚝배기계란찜,"계란 4개, 물 (계란과같은양), 당근, 부추, 파프리카다진거, 소금",[1. 우선 계란4개를 잘 풀어서 소금간을 해주고요~ 계란물을 최대한 많이 저어줘서...
1,브로콜리새우샐러드,"브로콜리 1.5컵, 칵테일새우 1컵, 소금 약간, 후춧가루 약간, 우유 7큰술, 올...",[1. 브로콜리는 먹기좋은 크기로 떼어내 씻어준후 끓는물에 소금을 조금 넣고 살짝 ...
2,모닝빵샌드위치,"모닝빵 3개, 감자 중간크기 2개, 당근 1/4개, 계란 3개, 양파 중간크기 1/...","[1. 재료: 모닝빵 ,감자 중간크기2개 당근1/4 ,계란3개, 양파 중간크기 1/..."
3,짬뽕스파게티,"물 3.5컵 , 건 홍고추 2개, 다시멸치 15개, 다시마 2장 (5X5cm), 스...","[1. 냄비에 물을 넉넉히 담고 소금 1수저와 올리브유 1수저를 넣어 끓인뒤, 면 ..."
4,칠리치즈감자튀김,"후렌치후라이 300g, 양파 1/2개, 돼지고기 다진것 200g, 치즈 1조각, 다...","[1. 냉동감자는 두번 튀겨내고 키친타월로 기름을 제거해 주세요., 2. 달궈진 팬..."
...,...,...,...
492,애플쿠키,No ingredients available,[No recipe available]
493,참치야채전,"애호박 1개, 양파 1/2개, 땡초 5개, 부침가루 3큰술, 계란 3개, 캔참치 1...","[1. 애호박,양파 잘게 채 썰어 볼에 담고 기름기 제거한 참치 담아준다, 2. ..."
494,닭튀김,"닭볶음탕용 닭 1마리, 튀김가루, 식용유 600ml, 우유 200ml, 맛소금 1/...",[1. 닭은 깨끗하게 씻어서 준비해 주세요. 살이 좀 두터운 부분은 잘익도록 칼집도...
495,초코볼쿠키,"밀가루 박력분 220g, 버터 135g, 설탕 90g, 우유 50g, 코코아가루 1...","[1. 코코아가루,박력분,베이킹파우더를 체쳐주세요, 2. 버터를 실온에녹인후에 크림..."


In [36]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[3001:3500]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df10 = pd.DataFrame(results)

No structured data found for 고구마피자


In [37]:
result_df10

,name,ingredients,recipe
0,애플시나몬소보로머핀,"달걀 2개, 박력분 80g, 베이킹 파우더 1/2작은술, 우유 30-50g, 사과 ...",[1. 토핑용 사과는 3-5mm 정도의 두께로 삼각형 모양으로 잘라서 준비해준다. ...
1,보쌈삶는법,"앞다리 사태살 1kg, 물 25컵 (5000CC), 무 1/4개, 양파 1/2개, ...",[1. 개인적으로 수육하면 앞다리 사태살이 좋긴 한데.. 없더라구요 ㅠㅠ 김장철이라...
2,콩조림,"콩, 다시마, 물, 간장, 물엿, 맛술",[1. 재료 : 서리태 서리태는 밥할 때 넣어서 먹어도 맛있고 콩조림 해서 먹어도 ...
3,전복닭고기버터구이,"닭고기 가슴살 300g, 전복 5개, 버터 3큰술, 다진 마늘 1큰술, 청주 1큰술...","[1. 닭고기 가슴살은 핏물을 뺀 다음 한 입 크기로 썰고, 다진 마늘, 청주, 설..."
4,흑미유부초밥,"흑미 1/2컵, 쌀 2컵, 유부 20개, 날치알 조금, 소금 2큰술, 물 1컵, 소...","[1. 흑미와 쌀을 준비하여 고슬하게 밥을 짓고, 유부는 절반으로 잘라 놓습니다.,..."
...,...,...,...
493,동태만두전골,No ingredients available,"[1. 무우하고 멸치를 냄비에 볶다가 물은 넣고 한소금 끓어주세요., 2. 육수는 ..."
494,오향족발,"돼지족 2개, 인스턴트커피 조금, 통후추 10알, 월계수잎 10장, 소주 1컵, 물...",[1. 족발을 물에 담궈 핏물을 빼주세요 만약 냉동 되어 있었다면 더 긴 시간 해동...
495,오향장육,"돼지고기삼겹살 800g (편육용), 마늘 6쪽, 생강 1쪽, 통후추 10알, 정향 ...","[1. 돼지고기는 찬물에 담궈 핏물을 빼 주세요~, 2. 냄비에 물을 넣고 대파 1..."
496,딸기팬케이크,"핫케이크가루, 우유, 딸기 10개, 올리고당 (딸기맛 또는시럽)",[No recipe available]


In [43]:
result_df
result_df2
result_df3
result_df4
result_df5
result_df6
# result_df7
# result_df8
# result_df9
# result_df10

,name,ingredients,recipe
0,딸기잼쿠키,"무염버터 100g, 설탕 60g, 노른자 1개, 박력분 145g, 옥수수전분 5g,...",[1. 볼에 말랑한 버터를 넣고 부드럽게 풀어준 다음 설탕을 넣고 충분히 섞어주세요...
1,동부묵,"동부가루 30g, 물 400cc, 호박가루, 녹차가루, 파프리카가루, 진간장, 다진...","[1. 동부가루를 30~31g 정도 전자저울에서 재었어요., 2. 물 400cc(종..."
2,부추고기만두,"돼지고기 1kg, 부추 1단, 양파 1-1/2개, 대파 4-5개, 다진마늘 2스푼,...","[1. 부추,대파,양파를 잘게 다집니다., 2. 돼지고기에 소금,후추,생강술로 밑간..."
3,단호박푸딩,"우유 300ml, 생크림 100ml, 단호박 으깬것 350g , 설탕 70g , 바...",[1. 먼저 카라멜을 만들어 주세요. 냄비에 설탕을 넣고 약한불로 끓여 가장자리부터...
4,소시지달걀말이,"소시지 바 2개, 달걀 5개, 슬라이스치즈 2장, 다진당근 1숟가락, 다진대파 1숟...","[1. 소시지바는 꼬치를 빼 준비한다., 2. 볼에 달걀을 풀고 다진당근, 다진대파..."
...,...,...,...
293,야채케이크,"쌀가루박력분 160g, 양파 1/2개, 콘옥수수 4큰술, 슬라이스햄 4장, 청양고추...",[1. 양파를 썰어주실 때 양파의 씹는맛을 더 느끼고 싶으시면 적당하게 썰어주셔도 ...
294,삼겹살김치볶음밥,"삼겹살 2줄, 김치 2/3밥공기, 대파 10cm , 마늘 4쪽, 밥 1.5공기 , ...",[1. (2인분 기준 입니다^~^) 대파10센티는 송송~ 썰어 주시구요~ 마늘 4쪽...
295,모카비스킷,"박력분 200g, 무염버터 40g, 설탕 30g, 소금 2g, 달걀 1개 50g, ...",[1. 먼저 볼에 초콜릿을 넣고 뜨거운물에 중탕하여 녹인 후 따뜻하게 데운 우유와 ...
296,고구마케익,"고구마 4개 (150g), 달걀흰자 3개, 우유 2스푼","[1. 찐고구마,군고구마 등을 으깨어주세요, 2. 달걀 흰자를 이용해 머랭을 만들어..."


In [44]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[3501:4000]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df11 = pd.DataFrame(results)

No structured data found for 진저쿠키
JSON parsing error: Invalid control character at: line 23 column 1985 (char 2792)
JSON parsing error: Invalid control character at: line 23 column 1985 (char 2792)


In [45]:
result_df11

,name,ingredients,recipe
0,참치버섯완자전,"캔참치(소) 1개, 만가닥버섯 약간, 청 피망 약간, 홍 피망 약간, 양파 약간, ...","[1. 만가닥버섯은 밑동 자르고 잘게 썰고, 청.홍피망, 양파도 잘게 썰어 준비하세..."
1,오뎅콩나물잡채,"콩나물, 당근 적당량, 오뎅 적당량, 일반간장 3숟가락, 참기름 1숟가락, 참기름 ...",[1. 오늘은 오뎅을 길게 썰어 당면을 대신하니 한장을 곱게 접어 한꺼번에 칼국수 ...
2,해물만두,"오징어 2마리, 새우 20~30마리, 부추 1줌, 두부 1모, 표고버섯 3개, 달걀...","[1. 새우, 오징어는 깨끗하게 손질한후 다져주세요. 믹서로 갈아도 되겠지만,,씹는..."
3,검은콩자반,"검은콩 2컵, 물 3컵, 식용유 1T, 간장 8T, 올리고당 6T, 맛술 2T, 설...","[1. 검은콩(서리태)을 씻은후 소쿠리에 받쳐 물기를 빼줍니다., 2. 냄비에 씻은..."
4,황태죽,"찬밥 1공기, 황태포 100g, 당근 다진것 1줌, 물 6컵, 들기름 1큰술, 소금...",[1. 준비하는 재료또한 참 간단하죠? 당근이 들어갔는데요. 채소는 원하는 좋아하는...
...,...,...,...
491,섬초무침,"섬초(시금치) 600g, 된장 3숟가락, 다진마늘 1숟가락, 참깨(깨소금) 1숟가락...",[1. 시금치 손질은 입맛에 따라서 조금씩 달리하는게 좋아요. 밑둥이 씹히는 ...
492,해물고추장불고기,"돼지고기 불고기용 600g, 대파 1대, 앙파 1개, 청양고추 2개, 마늘 5개, ...","[1. 8가지 모듬해물과 돼지고기를 적당한 크기로 잘라서 볼에 담습니다., 2. 마..."
493,시금치갈릭스파게티,"스파게티 면 120g, 시금치 70g, 마늘 5쪽, 포도씨오일 1~2큰술, 면 삶은...",[1. 스파게티 면은 끓는물에 소금을 조금 넣고 면을 넣고 7~9분정도 삶아서 찬물...
494,시금치호박된장국,"된장 2큰술, 다진마늘 0.5큰술, 시금치 2~3줌, 대파 1대, 다시마 1개 (5...",[1. 물 4컵에 다시멸치와 다시마를 넣고 팔팔 끓여주세요. 물이 끓기 시작하면 ...


In [49]:
# 여러 데이터프레임을 리스트에 저장
dataframes = [result_df, result_df2, result_df3, result_df4, result_df5, result_df6, result_df7, result_df8, result_df9, result_df10, result_df11]

# 데이터프레임을 하나로 합치기
combined_df = pd.concat(dataframes, ignore_index=True)

# 합쳐진 데이터프레임 확인
combined_df

,name,ingredients,recipe
0,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...","[1. 파프리카와 양파는 채썰어 살짝 볶아둔다., 2. 당면은 3분정도 삶아 물기를..."
1,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",[1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간...
2,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","[1. '날치알' 하나면 영양&비주얼 동시 캐치!, 2. 단무지, 맛살을 잘게 다져..."
3,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간","[1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다., 2. 호두는 끓는 물..."
4,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",[1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고...
...,...,...,...
3969,섬초무침,"섬초(시금치) 600g, 된장 3숟가락, 다진마늘 1숟가락, 참깨(깨소금) 1숟가락...",[1. 시금치 손질은 입맛에 따라서 조금씩 달리하는게 좋아요. 밑둥이 씹히는 ...
3970,해물고추장불고기,"돼지고기 불고기용 600g, 대파 1대, 앙파 1개, 청양고추 2개, 마늘 5개, ...","[1. 8가지 모듬해물과 돼지고기를 적당한 크기로 잘라서 볼에 담습니다., 2. 마..."
3971,시금치갈릭스파게티,"스파게티 면 120g, 시금치 70g, 마늘 5쪽, 포도씨오일 1~2큰술, 면 삶은...",[1. 스파게티 면은 끓는물에 소금을 조금 넣고 면을 넣고 7~9분정도 삶아서 찬물...
3972,시금치호박된장국,"된장 2큰술, 다진마늘 0.5큰술, 시금치 2~3줌, 대파 1대, 다시마 1개 (5...",[1. 물 4컵에 다시멸치와 다시마를 넣고 팔팔 끓여주세요. 물이 끓기 시작하면 ...


In [50]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[4001:4500]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df12 = pd.DataFrame(results)

JSON parsing error: Expecting ',' delimiter: line 12 column 19 (char 382)


In [51]:
result_df12

,name,ingredients,recipe
0,카라멜케이크,"박력분 150g, 버터 90g, 설탕 60g, 달걀 2개 (90g), 베이킹 파우더...",[1. 먼저 카라멜 크림을 만듭니다. 도톰한 소스팬에 설탕을 넣고 약불에서 서서히 ...
1,더덕무침,"더덕 11개, 고추장 3스푼 (소복하게), 고춧가루 1/2스푼, 올리고당 1스푼, ...","[1. 더덕은 손질해서 요렇게 하얀속살만 준비하세요. 더덕이 11개입니다., 2. ..."
2,콜라비생채보쌈,"콜라비 1개, 간장 2큰술, 다진 마늘 2큰술, 다진 생강 0.3큰술, 청주 1큰술...","[1. 깨끗이 씻은 콜라비는 잎과 줄기는 따로 떼고 껍질 벗겨주세요., 2. 채를 ..."
3,땅콩볶음,"땅콩 1종이컵, 식용유 2숟가락, 꽃소금 1숟가락, 설탕 1숟가락","[1. 땅콩을 종이컵 한가득 준비해주세요., 2. 후라이팬에 식용유 2숟가락을 넣고..."
4,오이채무침,"가시오이 1개, 고춧가루 1T, 국간장 1T, 참기름 1T, 통깨",[1. 가시오이 한개는 소금으로 박박 문질러 씻은 후 주방필러로 오이가시부분을 제거...
...,...,...,...
493,차돌박이양념구이,"차돌박이 300g, 양파 1/2개, 솔부추 50g, 양배추잎 4장, 상추 적당히, ...","[1. 맛난 차돌박이를 준비해 주시구요., 2. 양념장을 만듭니다. 요기 가운데 얼..."
494,오이달래무침,"손질한달래 70g, 오이 1/2개, 양파 1/8개, 고춧가루 3숟가락, 양조간장 2...","[1. 손질한 달래는 4cm정도로 자른다., 2. 오이는 길이로 반을 썰어 어슷썬다..."
495,열무된장지짐,"신 열무 400g, 국물용멸치 5마리, 들기름 5큰술, 고추가루 1큰술, 된장 1큰...",[1. 재료준비 해주세요 ★신 열무 400g 은 물에 2~3번 헹구어 물기를 꽉 짜...
496,샐러드피자,"새송이버섯 1개, 양파 1/2개, 피망 1개, 마늘 5개, 토마토소스, 바질가루, ...","[1. 각각의 음식에 맞게 재료를 준비해 줍니다, 2. 로메인과 양상추는 먹기좋게 ..."


In [54]:
results = []

for food_name in filtered_df['CKG_NM'].iloc[4501:5000]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df13 = pd.DataFrame(results)

JSON parsing error: Invalid control character at: line 5 column 21 (char 84)
JSON parsing error: Expecting ',' delimiter: line 23 column 989 (char 1699)
JSON parsing error: Invalid control character at: line 5 column 26 (char 89)
No structured data found for 동치미물냉면


In [55]:
result_df13

,name,ingredients,recipe
0,딸기롤샌드위치,"딸기 1팩, 딸기잼 30g, 크림치즈 1/2통, 식빵 5~6장","[1. 딸기는 초록색 꼭지를 칼로 잘라준뒤, 물에 깨끗이씻어 물기를 빼주세요~, 2..."
1,오이김치김밥,"밥 2공기, 김 2-3장, 오이 2개, 김치 2줌, 단무지 자른거 2-3개, 우엉 ...","[1. 오이를 소금으로 깨끗이 씻은다음 필러로 밀은후에, 2. 키친타올로 물기를 제..."
2,초콜릿진저브레드,"플레인초콜릿 다진것 150g, 무염버터 150g (실온에 둔), 달걀 3개 (흰자 ...",[1. 플레인초콜릿은 내열그릇에 담고 펄펄 끓지않는 뜨거운물이 담긴 냄비에 올려 서...
3,딸기사과무쌈롤,"무쌈 1통 (약 20장), 사과 1/3개, 딸기 7개, 당근 1/3개, 햄 50g,...","[1. 무순은 흐르는 물에 씻어주고, 당근과 사과는 적당한 굵기로 채썰어서 준비하세..."
4,머위쌈밥,"머위 20장, 굵은소금 1/2수저, 현미밥2공기, 된장 2수저, 매실액 1/2수저,...","[1. 손바닥보다 조금 큼지막한 머위를 준비했어요. 머위가 아직은 쓴맛도 덜하고, ..."
...,...,...,...
490,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간","[1. 파와 당근을 다져줍니다., 2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다..."
491,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...","[1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요., 2. 다음은 생크림과 물엿..."
492,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",[1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 ...
493,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",[1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그...


In [56]:
# 여러 데이터프레임을 리스트에 저장
dataframes2 = [result_df12, result_df13]

# 데이터프레임을 하나로 합치기
combined_df2 = pd.concat(dataframes2, ignore_index=True)

# 합쳐진 데이터프레임 확인
combined_df2

,name,ingredients,recipe
0,카라멜케이크,"박력분 150g, 버터 90g, 설탕 60g, 달걀 2개 (90g), 베이킹 파우더...",[1. 먼저 카라멜 크림을 만듭니다. 도톰한 소스팬에 설탕을 넣고 약불에서 서서히 ...
1,더덕무침,"더덕 11개, 고추장 3스푼 (소복하게), 고춧가루 1/2스푼, 올리고당 1스푼, ...","[1. 더덕은 손질해서 요렇게 하얀속살만 준비하세요. 더덕이 11개입니다., 2. ..."
2,콜라비생채보쌈,"콜라비 1개, 간장 2큰술, 다진 마늘 2큰술, 다진 생강 0.3큰술, 청주 1큰술...","[1. 깨끗이 씻은 콜라비는 잎과 줄기는 따로 떼고 껍질 벗겨주세요., 2. 채를 ..."
3,땅콩볶음,"땅콩 1종이컵, 식용유 2숟가락, 꽃소금 1숟가락, 설탕 1숟가락","[1. 땅콩을 종이컵 한가득 준비해주세요., 2. 후라이팬에 식용유 2숟가락을 넣고..."
4,오이채무침,"가시오이 1개, 고춧가루 1T, 국간장 1T, 참기름 1T, 통깨",[1. 가시오이 한개는 소금으로 박박 문질러 씻은 후 주방필러로 오이가시부분을 제거...
...,...,...,...
988,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간","[1. 파와 당근을 다져줍니다., 2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다..."
989,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...","[1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요., 2. 다음은 생크림과 물엿..."
990,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",[1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 ...
991,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",[1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그...


In [57]:
final_data = pd.concat([combined_df, combined_df2], ignore_index=True)

final_data

,name,ingredients,recipe
0,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...","[1. 파프리카와 양파는 채썰어 살짝 볶아둔다., 2. 당면은 3분정도 삶아 물기를..."
1,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",[1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간...
2,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","[1. '날치알' 하나면 영양&비주얼 동시 캐치!, 2. 단무지, 맛살을 잘게 다져..."
3,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간","[1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다., 2. 호두는 끓는 물..."
4,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",[1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고...
...,...,...,...
4962,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간","[1. 파와 당근을 다져줍니다., 2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다..."
4963,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...","[1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요., 2. 다음은 생크림과 물엿..."
4964,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",[1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 ...
4965,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",[1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그...


### recipe   
[] , \n, 'No recipe available' 삭제

In [58]:
final_data['recipe'] = final_data['recipe'].apply(lambda x: '\n'.join(x).replace('[','').replace(']',''))

In [59]:
filtered_df = filtered_df.iloc[0:5000]

In [62]:
# 컬럼명 변경
filtered_df = filtered_df.rename(columns={'CKG_NM': 'name'})

# 변경된 데이터프레임 확인
filtered_df.head()

,CKG_STA_ACTO_NM,name
0,간식,어묵김말이
1,일상,두부새우전
2,일상,알밥
3,일상,현미호두죽
4,술안주,북어갈비


In [63]:
# 'name' 열을 기준으로 데이터프레임 합치기
merged_df = pd.merge(filtered_df, final_data, on='name', how='inner')

# 합쳐진 데이터프레임 확인
merged_df

,CKG_STA_ACTO_NM,name,ingredients,recipe
0,간식,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...",1. 파프리카와 양파는 채썰어 살짝 볶아둔다.\n2. 당면은 3분정도 삶아 물기를 ...
1,일상,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간해...
2,일상,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","1. '날치알' 하나면 영양&비주얼 동시 캐치!\n2. 단무지, 맛살을 잘게 다져주..."
3,일상,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간",1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다.\n2. 호두는 끓는 물에...
4,술안주,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고 ...
...,...,...,...,...
4962,초스피드,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간",1. 파와 당근을 다져줍니다.\n2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다....
4963,간식,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...",1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요.\n2. 다음은 생크림과 물엿(...
4964,간식,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 설...
4965,일상,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그냥...


In [64]:
def clean_recipe(recipe_str):
    # 숫자와 점(예: '1.', '2.')을 제거하고, 줄바꿈 문자를 모두 제거하는 정규 표현식
    cleaned = recipe_str.replace('\n', ' ') # 줄바꿈 문자를 공백으로 대체
    return cleaned

In [65]:
merged_df['recipe'] = merged_df['recipe'].apply(clean_recipe)

In [66]:
merged_df

,CKG_STA_ACTO_NM,name,ingredients,recipe
0,간식,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...",1. 파프리카와 양파는 채썰어 살짝 볶아둔다. 2. 당면은 3분정도 삶아 물기를 뺀...
1,일상,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간해...
2,일상,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","1. '날치알' 하나면 영양&비주얼 동시 캐치! 2. 단무지, 맛살을 잘게 다져주세..."
3,일상,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간",1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다. 2. 호두는 끓는 물에 ...
4,술안주,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고 ...
...,...,...,...,...
4962,초스피드,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간",1. 파와 당근을 다져줍니다. 2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다. ...
4963,간식,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...",1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요. 2. 다음은 생크림과 물엿(요...
4964,간식,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 설...
4965,일상,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그냥...


In [67]:
merged_df = merged_df[~merged_df['recipe'].apply(lambda x: 'No recipe available' in x)].reset_index(drop=True)

In [68]:
merged_df

,CKG_STA_ACTO_NM,name,ingredients,recipe
0,간식,어묵김말이,"구멍어묵 4개, 김밥김 2장, 당면 20g, 양파 1/4개, 파프리카 1/4개, 달...",1. 파프리카와 양파는 채썰어 살짝 볶아둔다. 2. 당면은 3분정도 삶아 물기를 뺀...
1,일상,두부새우전,"손두부 1모, 새우 20g, 부추 10g, 달걀 1개, 소금 약간, 부침유 적당히,...",1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간해...
2,일상,알밥,"쌀밥 2공기, 날치알 2큰술, 맛살 2줄, 다진단무지 4큰술, 어린잎채소 2줌, 도...","1. '날치알' 하나면 영양&비주얼 동시 캐치! 2. 단무지, 맛살을 잘게 다져주세..."
3,일상,현미호두죽,"현미 4컵, 찹쌀 2컵, 호두 50g, 물 1/2컵, 소금 약간",1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다. 2. 호두는 끓는 물에 ...
4,술안주,북어갈비,"북어포 1마리, 찹쌀가루 1C, 간장 2T, 설탕 1T, 물 1T, 다진파 1T, ...",1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고 ...
...,...,...,...,...
4843,초스피드,새송이버섯부침,"새송이버섯, 파, 당근, 계란, 소금 약간",1. 파와 당근을 다져줍니다. 2. 새송이버섯은 먹기좋은 크기로 잘라 준비합니다. ...
4844,간식,초콜릿생크림머핀,"생크림 120g, 유정란 1개, 유기농설탕 70g, 요리당 20g, 우리밀 120g...",1. 우선 유정란 1개와 설탕을 볼에넣고 잘 섞어요. 2. 다음은 생크림과 물엿(요...
4845,간식,씨리얼쿠키,"버터 90g, 박력분 100g, 설탕 40g, 베이킹파우더 2g, 아몬드 60g, ...",1. 버터는 실온에 미리 꺼내어두세요. 제가 사용한 애플올리고 씨리얼은 달콤해서 설...
4846,일상,미나리오이무침,"미나리 1단, 오이 1개, 당근, 고추장 1T, 고춧가루 2T, 설탕 2T, 매실액...",1. 노란 떡잎이..... 손질해서 몇 번을 씻어냈어요. 깨끗하게 손질을 해서 그냥...


### ingredients


In [70]:
import re
def clean_ingredients(ingredient_str):
    # 숫자+한글, 숫자+영어, "1/2개" 등 제거하는 정규 표현식
    cleaned = re.sub(r'\d+/\d+[가-힣]*|\d+[a-zA-Z가-힣]*|약간|적당히', '', ingredient_str)
    # 쉼표로 나누고 공백 제거
    cleaned_list = [item.strip() for item in cleaned.split(',') if item.strip()]
    return ', '.join(cleaned_list)

In [71]:
merged_df['ingredients'] = merged_df['ingredients'].apply(clean_ingredients)

In [72]:
merged_df.head()

,CKG_STA_ACTO_NM,name,ingredients,recipe
0,간식,어묵김말이,"구멍어묵, 김밥김, 당면, 양파, 파프리카, 달걀, 튀김가루, 카놀라유 적당량, 간...",1. 파프리카와 양파는 채썰어 살짝 볶아둔다. 2. 당면은 3분정도 삶아 물기를 뺀...
1,일상,두부새우전,"손두부, 새우, 부추, 달걀, 소금, 부침유, 참기름",1. 두부는 먹기 좋은 크기로 직사각형으로 1Cm정도을 두께로 썰어 소금으로 밑간해...
2,일상,알밥,"쌀밥, 날치알, 맛살, 다진단무지, 어린잎채소, 도시락 김, 참기름, 통깨, 다진김...","1. '날치알' 하나면 영양&비주얼 동시 캐치! 2. 단무지, 맛살을 잘게 다져주세..."
3,일상,현미호두죽,"현미, 찹쌀, 호두, 물, 소금",1. 현미와 찹쌀은 씻어서 불린 후 분말기로 적당히 간다. 2. 호두는 끓는 물에 ...
4,술안주,북어갈비,"북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루",1. 북어포를 준비해주세여.머리랑 꼬리는 잘라주시고 지느러미도 정리해주세요.그리고 ...


In [73]:
merged_df['recipe'][0]

'1. 파프리카와 양파는 채썰어 살짝 볶아둔다. 2. 당면은 3분정도 삶아 물기를 뺀 뒤 볶아둔 채소와 간장, 참기름을 넣어 버무린다. 튀김가루 1/2컵에 물 2/3컵을 넣고 섞어 튀김옷을 만든다. 3. 원통형 어묵에 양념한 당면 소를 채워넣는다. 4. 4등분한 김에 튀김옷을 얇게 바른 뒤 어묵을 올려 김밥처럼 돌돌 만다. 5. 완성된 어묵 김말이를 튀김옷에 담갔다 건져 170C의 카놀라유에 노릇하게 튀겨낸다. 6. 기름기를 뺀 어묵 김말이를 먹기 좋게 썰어 접시에 담는다.'

In [74]:
merged_df.to_csv('C:/Users/KimDongyoung/Desktop/my_git/mygit/DAT/24-2/프로젝트/data/recipe_info.csv', index=False)